# RADARSAT

In [ ]:
import geopandas as gpd
import pandas as pd
import cartopy
import matplotlib.pyplot as plt
%matplotlib inline
import json
import shapely

import shared_functions as sf

# settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 150)

## Functions

In [ ]:
def annual_summary(df, size_col = 'size_GB'):
    count_size = (
        (df.groupby(['year']).agg(
        count = ('id','count'),
        size = (size_col, 'sum'),
        size_MB = ('size_MB','sum'),
        ))
        .reset_index()
        #.pivot(index='year', values=['count','size'])
        .fillna(0)
    )
    count_size.loc["Total"] = count_size.sum()
    count_size['count'] = count_size['count'].astype(int)
    count_size['mean_size_MB'] = count_size['size_MB']/count_size['count'] 
    count_size[size_col] = (count_size['size']).round(2)
    count_size = count_size.drop(columns=['size'])
    return count_size

## RADARSAT-1 Level 1

In [ ]:
with open('metadata/RADARSAT-1_LEVEL1_-50N_products.json') as f:
    data = json.load(f)
df = pd.DataFrame.from_dict(data, orient='index')
df = sf.preprocess_cmr_df(df, crs=3031, lon_first=True, aoi_filter=True)
df.head(2)

In [ ]:
df.to_crs(4326).geometry.bounds.miny

In [ ]:
df.to_crs(4326).time_start.max()

In [ ]:
annual_summary(df, size_col='size_GB')

In [ ]:
sf.plot_results_footprint_map(df)

In [ ]:
sf.plot_timeseries_products(df, 
	title=f'RADARSAT-1 (Level 1A) - Weekly Products' ,
	stack_col='dataset_id', 
	date_col='time_start',
	count_freq='7D', 
    plot_freq='2M')

In [ ]:
sf.plot_frequency(df[df['year']==2000],
                  title = 'RADARSAT-1 (2008)',
                  shape=(1000,1000),
                  #force_max=30
                )

In [ ]:
sf.plot_multiple_frequency(
    df,
    title = f"RADARSAT-1 Revisit Frequency by Year",
    group='year', 
    sort_group='year',
    shape=(1000,1000),
    cbar_label='Revisit Count',
    n_cols=5,
)
# dates and coverage lines up with - https://asf.alaska.edu/data-sets/derived-data-sets/radarsat-antarctic-mapping-project-ramp/

## RADARSAT-2

In [ ]:
with open('metadata/Radarsat2RawProducts_-50N_products.json') as f:
    data = json.load(f)
df = pd.DataFrame.from_dict(data, orient='index')
#df = sf.preprocess_cmr_df(df, crs=3031, lon_first=True, aoi_filter=True)
df['geometry'] = df['wktGeometry'].apply(lambda x : shapely.wkt.loads(x))
df = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:3031")

In [ ]:
sf.plot_results_footprint_map(df)

In [ ]:
df['wktGeometry'].iloc[0]

In [ ]:
df.plot()